In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
for i in application_df.columns:
    print(i, len(application_df[i].unique()))

APPLICATION_TYPE 17
AFFILIATION 6
CLASSIFICATION 71
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 8747
IS_SUCCESSFUL 2


In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_type_totals = application_df['APPLICATION_TYPE'].value_counts()
app_type_totals

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_totals[app_type_totals < 700].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts.head(30)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
Name: CLASSIFICATION, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 700].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_df = pd.get_dummies(application_df)
numeric_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
# Split our preprocessed data into our features and target arrays
y = numeric_df['IS_SUCCESSFUL']
X = numeric_df.drop(['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=77)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
#Optimization Attempt #1
nn_model1 = tf.keras.models.Sequential()
nn_model1.add(tf.keras.layers.Dense(units=27, activation="tanh"))
nn_model1.add(tf.keras.layers.Dense(units=13, activation="relu"))
nn_model1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model1.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
804/804 [==============================] - 6s 4ms/step - loss: 0.5807 - accuracy: 0.7140
Epoch 2/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5561 - accuracy: 0.7295
Epoch 3/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5526 - accuracy: 0.7294
Epoch 4/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5499 - accuracy: 0.7312
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5486 - accuracy: 0.7337
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7317
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5467 - accuracy: 0.7348
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accuracy: 0.7341
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5453 - accuracy: 0.7341
Epoch 10/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5448 - accuracy: 0.7348

In [12]:
#Optimization attempt #2
nn_model2 = tf.keras.models.Sequential()
nn_model2.add(tf.keras.layers.Dense(units=40, activation="tanh"))
nn_model2.add(tf.keras.layers.Dense(units=25, activation="tanh"))
nn_model2.add(tf.keras.layers.Dense(units=19, activation="relu"))
nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model2.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
804/804 [==============================] - 6s 2ms/step - loss: 0.5702 - accuracy: 0.7181
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5547 - accuracy: 0.7293
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5512 - accuracy: 0.7306
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5484 - accuracy: 0.7324
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5475 - accuracy: 0.7320
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5468 - accuracy: 0.7338
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5457 - accuracy: 0.7344
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5454 - accuracy: 0.7327
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5451 - accuracy: 0.7345
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5439 - accuracy: 0.7350

In [13]:
#Optimization attempt #3
nn_model3 = tf.keras.models.Sequential()
nn_model3.add(tf.keras.layers.Dense(units=8, activation="tanh"))
nn_model3.add(tf.keras.layers.Dense(units=5, activation="tanh"))
nn_model3.add(tf.keras.layers.Dense(units=6, activation="tanh"))
nn_model3.add(tf.keras.layers.Dense(units=5, activation="relu"))
nn_model3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=5)


# Compile the Sequential model together and customize metrics
nn_model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model3.fit(X_train_scaled, y_train, epochs=30, callbacks=[checkpoint])

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/30
804/804 [==============================] - 3s 2ms/step - loss: 0.5976 - accuracy: 0.7018
Epoch 2/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5627 - accuracy: 0.7262
Epoch 3/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5577 - accuracy: 0.7295
Epoch 4/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5549 - accuracy: 0.7304
Epoch 5/30
800/804 [============================>.] - ETA: 0s - loss: 0.5531 - accuracy: 0.7304

804/804 [==============================] - 2s 2ms/step - loss: 0.5531 - accuracy: 0.7304
Epoch 6/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5517 - accuracy: 0.7320
Epoch 7/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5513 - accuracy: 0.7323
Epoch 8/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5506 - accuracy: 0.7324
Epoch 9/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5506 - accuracy: 0.7335
Epoch 10/30
784/804 [============================>.] - ETA: 0s - loss: 0.5496 - accuracy: 0.7319

804/804 [==============================] - 3s 3ms/step - loss: 0.5497 - accuracy: 0.7318
Epoch 11/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7325
Epoch 12/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5486 - accuracy: 0.7322
Epoch 13/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7321
Epoch 14/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7331
Epoch 15/30
797/804 [============================>.] - ETA: 0s - loss: 0.5476 - accuracy: 0.7326

804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7325
Epoch 16/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7318
Epoch 17/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5472 - accuracy: 0.7342
Epoch 18/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7334
Epoch 19/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5464 - accuracy: 0.7336
Epoch 20/30
802/804 [============================>.] - ETA: 0s - loss: 0.5463 - accuracy: 0.7334

804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7334
Epoch 21/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accuracy: 0.7339
Epoch 22/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7333
Epoch 23/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5460 - accuracy: 0.7324
Epoch 24/30
804/804 [==============================] - 3s 4ms/step - loss: 0.5452 - accuracy: 0.7325
Epoch 25/30
803/804 [============================>.] - ETA: 0s - loss: 0.5457 - accuracy: 0.7335

804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7336
Epoch 26/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5449 - accuracy: 0.7343
Epoch 27/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5452 - accuracy: 0.7334
Epoch 28/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5446 - accuracy: 0.7341
Epoch 29/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5445 - accuracy: 0.7337
Epoch 30/30
790/804 [============================>.] - ETA: 0s - loss: 0.5449 - accuracy: 0.7333

804/804 [==============================] - 2s 2ms/step - loss: 0.5447 - accuracy: 0.7335
268/268 - 0s - loss: 0.5544 - accuracy: 0.7262 - 456ms/epoch - 2ms/step
Loss: 0.5544193983078003, Accuracy: 0.7261807322502136


In [14]:
nn_model3.save("nn_models.h5")